In [1]:
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# training and evaluating traditional machine learning models 
# such as SVM, Random Forest, k-NN, and Logistic Regression.

# Define directories
train_dir = 'BoneFractureDataset/training'
test_dir = 'BoneFractureDataset/testing'

# Load and preprocess data
def load_data(directory):
    datagen = ImageDataGenerator(rescale=1./255)
    generator = datagen.flow_from_directory(
        directory,
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary',
        shuffle=False
    )

    num_samples = generator.samples
    num_classes = generator.num_classes
    class_labels = list(generator.class_indices.keys())

    data = np.zeros((num_samples, 224, 224, 3), dtype=np.float32)
    labels = np.zeros(num_samples, dtype=np.float32)

    i = 0
    for inputs_batch, labels_batch in generator:
        data[i * 32: (i + 1) * 32] = inputs_batch
        labels[i * 32: (i + 1) * 32] = labels_batch
        i += 1
        if i * 32 >= num_samples:
            break

    data = data.reshape((num_samples, -1))
    return data, labels, class_labels

# Extract features and labels from training and testing data
X_train, y_train, class_labels = load_data(train_dir)
X_test, y_test, _ = load_data(test_dir)

# Split training data for validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Define and train the models
models = {
    "SVM": SVC(kernel='linear', random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "k-NN": KNeighborsClassifier(n_neighbors=5),
    "Logistic Regression": LogisticRegression(random_state=42)
}

# Train and evaluate each model
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)
    y_test_pred = model.predict(X_test)
    
    print(f"{model_name} Model")
    print("Validation Accuracy:", accuracy_score(y_val, y_val_pred))
    print("Test Accuracy:", accuracy_score(y_test, y_test_pred))
    print("Classification Report (Test):")
    print(classification_report(y_test, y_test_pred))
    print("-" * 50)

# Function to test a single image
def test_single_image(image_path, models, scaler):
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img) / 255.0
    img_array = img_array.reshape(1, -1)
    img_array = scaler.transform(img_array)
    
    results = {}
    for model_name, model in models.items():
        prediction = model.predict(img_array)
        results[model_name] = class_labels[int(prediction[0])]
    
    return results




Found 8863 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
SVM Model
Validation Accuracy: 0.8753525098702764
Test Accuracy: 0.75
Classification Report (Test):
              precision    recall  f1-score   support

         0.0       0.86      0.70      0.77       360
         1.0       0.65      0.82      0.73       240

    accuracy                           0.75       600
   macro avg       0.75      0.76      0.75       600
weighted avg       0.77      0.75      0.75       600

--------------------------------------------------
Random Forest Model
Validation Accuracy: 0.9949238578680203
Test Accuracy: 0.71
Classification Report (Test):
              precision    recall  f1-score   support

         0.0       0.97      0.53      0.69       360
         1.0       0.58      0.97      0.73       240

    accuracy                           0.71       600
   macro avg       0.78      0.75      0.71       600
weighted avg       0.81      0.71      0.70       600

-

C:\Users\nsona\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Model
Validation Accuracy: 0.8522278623801467
Test Accuracy: 0.7383333333333333
Classification Report (Test):
              precision    recall  f1-score   support

         0.0       0.90      0.63      0.74       360
         1.0       0.62      0.90      0.73       240

    accuracy                           0.74       600
   macro avg       0.76      0.77      0.74       600
weighted avg       0.79      0.74      0.74       600

--------------------------------------------------


In [2]:

# Test a single image
image_path =    'BoneFractureDataset\\testing\\fractured\\1.jpg'

results = test_single_image(image_path, models, scaler)
print(f"Predictions for {image_path}:")
for model_name, prediction in results.items():
    print(f"{model_name}: {prediction}")

Predictions for BoneFractureDataset\testing\fractured\1.jpg:
SVM: fractured
Random Forest: not_fractured
k-NN: not_fractured
Logistic Regression: fractured


In [5]:
image_paths = [
    'BoneFractureDataset/testing/fractured/1.jpg',
    'BoneFractureDataset/testing/fractured/1-rotated1.jpg',
    'BoneFractureDataset/testing/fractured/1-rotated1-rotated1.jpg',
    'BoneFractureDataset/testing/fractured/1-rotated1-rotated1-rotated1.jpg',
    'BoneFractureDataset/testing/fractured/1-rotated1-rotated1-rotated2.jpg',
    'BoneFractureDataset/testing/fractured/1-rotated1-rotated1-rotated3.jpg',
    'BoneFractureDataset/testing/fractured/2.jpg'
]

for image_path in image_paths:
    results = test_single_image(image_path, models, scaler)
    print(f"\nPredictions for {image_path}:")
    for model_name, prediction in results.items():
        print(f"{model_name}: {prediction}")


Predictions for BoneFractureDataset/testing/fractured/1.jpg:
SVM: fractured
Random Forest: not_fractured
k-NN: not_fractured
Logistic Regression: fractured

Predictions for BoneFractureDataset/testing/fractured/1-rotated1.jpg:
SVM: fractured
Random Forest: not_fractured
k-NN: not_fractured
Logistic Regression: fractured

Predictions for BoneFractureDataset/testing/fractured/1-rotated1-rotated1.jpg:
SVM: fractured
Random Forest: not_fractured
k-NN: not_fractured
Logistic Regression: fractured

Predictions for BoneFractureDataset/testing/fractured/1-rotated1-rotated1-rotated1.jpg:
SVM: fractured
Random Forest: not_fractured
k-NN: not_fractured
Logistic Regression: fractured

Predictions for BoneFractureDataset/testing/fractured/1-rotated1-rotated1-rotated2.jpg:
SVM: fractured
Random Forest: not_fractured
k-NN: not_fractured
Logistic Regression: fractured

Predictions for BoneFractureDataset/testing/fractured/1-rotated1-rotated1-rotated3.jpg:
SVM: fractured
Random Forest: not_fractured
k